## Testing Loading Checkpoints

In [32]:
from zoobot.tensorflow.estimators import define_model, preprocess
import tensorflow as tf
from tensorflow.keras import layers

In [12]:
finetuned_location = 'C:/Users/oryan/Documents/zoobot_new/model-logs/2022-05-05-HEC-all-unfrozen-0.89/finetuned/checkpoint' # - Doesn't work
# finetuned_location = 'C:/Users/oryan/Documents/zoobot_tests/model/checkpoint' # - Works
# finetuned_location = r'C:\Users\oryan\Documents\esac-project\logs\top_block7_block6_unfrozen\trained-head\checkpoint' # Doesn't work. Not even trained head works...

In [29]:
folder = 'C:/Users/oryan/Documents/zoobot_new/pretrained-models'
pretrained_checkpoint = f'{folder}/replicated_train_only_greyscale_tf/replicated_train_only_greyscale_tf/checkpoint'

In [30]:
requested_img_size = 300
crop_size = int(0.75 * requested_img_size)
resize_size = 224

In [33]:
base_model = define_model.load_model(
    pretrained_checkpoint,
    include_top=False,
    input_size = requested_img_size,
    crop_size = crop_size,
    resize_size = resize_size,
    output_dim=None,
    channels=1,
    expect_partial=True
)

base_model.trainable = False

para_relu = layers.PReLU()

new_head = tf.keras.Sequential([
    layers.InputLayer(input_shape=(7,7,1280)),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64,activation=para_relu),
    #layers.Dropout(0.75),
    layers.Dense(64,activation='elu'),
    layers.Dropout(0.25),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(64,activation = para_relu),
    layers.Dropout(0.25),
    layers.Dense(1,activation='sigmoid',name='sigmoid_output')
])

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(requested_img_size, requested_img_size,1)),
    base_model,
    new_head
])

In [39]:
model.load_weights(finetuned_location).expect_partial()